### Access BR3 item data

In [1]:
from dataminer.connector import BigRed3
from dataminer.utils import get_secret

# establish connection
p = get_secret('p')
br3 = BigRed3(password=p)

# write your sql statement
sql_brand = "SELECT DISTINCT brand_n FROM prd_itm_fnd.item WHERE (brand_n IS NOT NULL) AND (brand_n != '') AND (brand_n != 'No Brand') AND (brand_n != 'No Name')"

# set parameters
br3.execute('''set hive.execution.engine = tez''')

# run query and view data
df_brandname = br3.query(sql_brand)
df_brandname.head()

p: ········
2023-02-22 17:25:58,309 | INFO | dataminer : Connecting user z00c01y


,brand_n
0,#Boneless
1,1 Badd Ride
2,1 Basics Made Simple
3,1 Million
4,10 Barrel Brewing Co.


In [5]:
# save data to csv

# df.to_csv('distinct_brand_name.csv', index=False)

### Dictionary part 1 - common English words

In [2]:
# load nltk package

# pip install nltk
import nltk

In [11]:
# download English words

nltk.download('brown')
from nltk.corpus import brown

brown_nltk = brown.words()
english_dictionary_lst = []
for ele in brown_nltk:
    # make all lowercases
    ele_lower = ele.lower()
    # remove potential duplicates
    if ele_lower not in english_dictionary_lst:
        english_dictionary_lst.append(ele_lower)
    else:
        pass
english_dictionary_lst[:10]

[nltk_data] Downloading package brown to /home/dataminer/nltk_data...
[nltk_data]   Package brown is already up-to-date!


['the',
 'fulton',
 'county',
 'grand',
 'jury',
 'said',
 'friday',
 'an',
 'investigation',
 'of']

In [12]:
# check current list length

len(english_dictionary_lst)

49815

### Dictionary part 2 - Target products brand names

In [19]:
# using the distinct brand names from item table

tgt_brand_n_lst = df_brandname['brand_n'].values.tolist()
tgt_brand_n_lst[:10]

['#Boneless',
 '1 Badd Ride',
 '1 Basics Made Simple',
 '1 Million',
 '10 Barrel Brewing Co.',
 '10 Strawberry Street',
 '1000toys',
 '103 Collection',
 '10th Avenue Tea',
 '11 Wells']

In [20]:
# make all brand name lower cases
tgt_brand_n_lst_lower = []
for bn in tgt_brand_n_lst:
    # make all lowercases
    bn_lower = bn.lower()
    # remove potential duplicates
    if bn_lower not in tgt_brand_n_lst_lower:
        tgt_brand_n_lst_lower.append(bn_lower)
    else:
        pass

# check current list length
len(tgt_brand_n_lst_lower)

37536

In [21]:
# appending brand names to english dictionary list

eng_brand_dictionary_lst = english_dictionary_lst.copy()
eng_brand_dictionary_lst.extend(tgt_brand_n_lst_lower)
eng_brand_dictionary_lst = list(set(eng_brand_dictionary_lst))
len(eng_brand_dictionary_lst)

85094

### Dictionary part 3 - Seperate brand names into individual words and save to dictionary as well

In [24]:
# split brand names into individual words

tgt_brand_word_lst = []
for n in tgt_brand_n_lst_lower:
    n_lst = n.split()
    tgt_brand_word_lst.extend(n_lst)

# remove duplicates
tgt_brand_word_lst = list(set(tgt_brand_word_lst))
tgt_brand_word_lst[:10]

['equil',
 'heywell',
 'standdesk',
 'biskit',
 'azulle',
 'duluth',
 'bender',
 'pinkie',
 'thy-hom',
 'ww']

In [26]:
# check current list length

len(tgt_brand_word_lst)

29760

In [27]:
# appending brand words to dictionary list

eng_brand_dictionary_lst.extend(tgt_brand_word_lst)
eng_brand_dictionary_lst = list(set(eng_brand_dictionary_lst))
len(eng_brand_dictionary_lst)

92603

### Export current dictionary to csv and store in database

In [30]:
import pandas as pd

In [31]:
# create a dataframe
dictionary_df = pd.DataFrame({'words':eng_brand_dictionary_lst})
dictionary_df.head()

,words
0,the flux capacity
1,equil
2,heywell
3,jerusalem
4,woke


In [35]:
dictionary_df.to_csv('dictionary_v1.csv', index=False, header=False)

In [37]:
dictionary_df.shape

(92603, 1)